[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/petals-colab/blob/main/petals_colab.ipynb)

In [ ]:
!pip install git+https://github.com/bigscience-workshop/petals

import torch
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM

MODEL_NAME = "enoch/llama-65b-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, add_bos_token=False)

model = AutoDistributedModelForCausalLM.from_pretrained(MODEL_NAME)
model = model.cuda()

In [ ]:
inputs = tokenizer('A cat in French is "', return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=3)
print(tokenizer.decode(outputs[0]))

In [ ]:
fake_token = tokenizer("^")["input_ids"][0]  # Workaround to make SentencePiece .decode() keep leading spaces

with model.inference_session(max_length=512) as sess:
    while True:
        prompt = input('Human: ')
        if prompt == "":
            break
        prefix = f"Human: {prompt}\nFriendly AI:"
        prefix = tokenizer(prefix, return_tensors="pt")["input_ids"].cuda()
        print("Friendly AI:", end="", flush=True)

        while True:
            outputs = model.generate(
                prefix, max_new_tokens=1, do_sample=True, top_p=0.9, temperature=0.75, session=sess
            )
            outputs = tokenizer.decode([fake_token, outputs[0, -1].item()])[1:]
            print(outputs, end="", flush=True)
            if "\n" in outputs:
                break
            prefix = None  # Prefix is passed only for the 1st token of the bot's response